In [ ]:
# Model Evaluation - Modèle de Prédiction de Rues Risquées

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Librairies pour l'évaluation
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, validation_curve, learning_curve
from sklearn.inspection import permutation_importance
import joblib

# Librairies pour visualisation avancée
try:
    import shap
    HAS_SHAP = True
except ImportError:
    HAS_SHAP = False
    print("⚠️ SHAP non installé - analyse d'interprétabilité limitée")

# Pour les cartes
import folium
from folium.plugins import HeatMap

print("📊 Début de l'évaluation du modèle")
print("="*60)

# =====================================================================
# 1. CHARGEMENT DU MODÈLE ET DES DONNÉES
# =====================================================================

print("\n📁 Chargement du modèle et des données...")

def load_model_and_data():
    """Charge le modèle entraîné et les données de test"""
    import json
    
    try:
        # Chargement des métadonnées du modèle
        with open('../models/model_metadata.json', 'r') as f:
            model_metadata = json.load(f)
        
        print(f"✅ Métadonnées du modèle chargées:")
        print(f"  - Modèle: {model_metadata['model_name']}")
        print(f"  - Type: {model_metadata['model_type']}")
        print(f"  - Features: {model_metadata['num_features']}")
        print(f"  - Date d'entraînement: {model_metadata['model_training_date']}")
        
        # Chargement du modèle
        if model_metadata['model_type'] == 'neural_network':
            try:
                import keras
                model = keras.models.load_model('../models/risk_prediction_model.h5')
                print(f"✅ Modèle Neural Network chargé")
            except ImportError:
                print("❌ TensorFlow non disponible pour charger le Neural Network")
                return None, None, None, None
        else:
            model = joblib.load('../models/risk_prediction_model.joblib')
            print(f"✅ Modèle {model_metadata['model_name']} chargé")
        
        # Chargement du scaler
        scaler = joblib.load('../models/feature_scaler.joblib')
        print(f"✅ Scaler chargé")
        
        # Chargement des données de features
        try:
            X_test = pd.read_parquet('../data/features/feature_matrix.parquet')
            y_test = pd.read_parquet('../data/features/target_variable.parquet')['risk_score']
            
            # Division simulée des données (car on n'a pas sauvé la division exacte)
            # On prend les 20% derniers comme test set
            test_size = int(0.2 * len(X_test))
            X_test_subset = X_test.iloc[-test_size:]
            y_test_subset = y_test.iloc[-test_size:]
            
            print(f"✅ Données de test chargées: {X_test_subset.shape[0]} observations")
            
        except FileNotFoundError:
            print("⚠️ Fichiers de features non trouvés - utilisation de données simulées")
            # Génération de données simulées pour la démo
            np.random.seed(42)
            n_samples = 20
            n_features = model_metadata['num_features']
            X_test_subset = pd.DataFrame(
                np.random.randn(n_samples, n_features),
                columns=model_metadata['feature_names'][:n_features]
            )
            y_test_subset = pd.Series(np.random.uniform(0, 10, n_samples))
            print(f"⚠️ Données simulées créées: {n_samples} observations")
        
        return model, scaler, X_test_subset, y_test_subset, model_metadata
        
    except FileNotFoundError as e:
        print(f"❌ Fichier non trouvé: {e}")
        print("   Vérifiez que le model development a été exécuté")
        return None, None, None, None, None
    except Exception as e:
        print(f"❌ Erreur de chargement: {e}")
        return None, None, None, None, None

# Chargement du modèle et des données
model, scaler, X_test, y_test, model_metadata = load_model_and_data()

# =====================================================================
# 2. ÉVALUATION DES PERFORMANCES GÉNÉRALES
# =====================================================================

print("\n" + "="*60)
print("📈 ÉVALUATION DES PERFORMANCES GÉNÉRALES")
print("="*60)

def comprehensive_evaluation(model, X_test, y_test, scaler, model_metadata):
    """Évaluation complète des performances du modèle"""
    
    print(f"\n🔬 Évaluation en cours...")
    
    # Normalisation des features
    X_test_scaled = scaler.transform(X_test)
    
    # Prédictions
    if model_metadata['model_type'] == 'neural_network':
        y_pred = model.predict(X_test_scaled, verbose=0).flatten()
    else:
        y_pred = model.predict(X_test_scaled)
    
    # Calcul des métriques
    metrics = {
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'MAE': mean_absolute_error(y_test, y_pred),
        'R²': r2_score(y_test, y_pred),
        'MAPE': np.mean(np.abs((y_test - y_pred) / y_test)) * 100,
        'Max Error': np.max(np.abs(y_test - y_pred))
    }
    
    # Calcul d'erreurs relatives
    relative_errors = np.abs(y_test - y_pred) / y_test
    metrics['Median Relative Error'] = np.median(relative_errors) * 100
    metrics['95th Percentile Error'] = np.percentile(relative_errors, 95) * 100
    
    print(f"\n📊 MÉTRIQUES DE PERFORMANCE:")
    print(f"  • RMSE (Root Mean Square Error): {metrics['RMSE']:.3f}")
    print(f"  • MAE (Mean Absolute Error): {metrics['MAE']:.3f}")
    print(f"  • R² (Coefficient de détermination): {metrics['R²']:.3f}")
    print(f"  • MAPE (Mean Absolute Percentage Error): {metrics['MAPE']:.1f}%")
    print(f"  • Erreur maximale: {metrics['Max Error']:.3f}")
    print(f"  • Erreur relative médiane: {metrics['Median Relative Error']:.1f}%")
    print(f"  • 95e percentile d'erreur: {metrics['95th Percentile Error']:.1f}%")
    
    # Interprétation des résultats
    print(f"\n🎯 INTERPRÉTATION:")
    if metrics['R²'] > 0.8:
        print("  ✅ Excellent: R² > 0.8 - Modèle très prédictif")
    elif metrics['R²'] > 0.6:
        print("  ✅ Bon: R² > 0.6 - Modèle assez prédictif")
    elif metrics['R²'] > 0.4:
        print("  ⚠️ Moyen: R² > 0.4 - Modèle modérément prédictif")
    else:
        print("  ❌ Faible: R² < 0.4 - Modèle peu prédictif")
    
    if metrics['MAPE'] < 15:
        print("  ✅ Erreur acceptable: MAPE < 15%")
    elif metrics['MAPE'] < 25:
        print("  ⚠️ Erreur modérée: MAPE < 25%")
    else:
        print("  ❌ Erreur importante: MAPE > 25%")
    
    return metrics, y_pred

# Évaluation si le modèle est chargé
if model is not None:
    performance_metrics, predictions = comprehensive_evaluation(model, X_test, y_test, scaler, model_metadata)

# =====================================================================
# 3. ANALYSE DES RÉSIDUS ET ERREURS
# =====================================================================

print("\n" + "="*60)
print("🔍 ANALYSE DES RÉSIDUS ET ERREURS")
print("="*60)

def residual_analysis(y_true, y_pred):
    """Analyse détaillée des résidus"""
    
    print(f"\n📊 Analyse des résidus...")
    
    # Calcul des résidus
    residuals = y_true - y_pred
    
    # Statistiques des résidus
    print(f"\n📈 STATISTIQUES DES RÉSIDUS:")
    print(f"  • Moyenne: {residuals.mean():.3f}")
    print(f"  • Écart-type: {residuals.std():.3f}")
    print(f"  • Minimum: {residuals.min():.3f}")
    print(f"  • Maximum: {residuals.max():.3f}")
    print(f"  • Skewness: {residuals.skew():.3f}")
    print(f"  • Kurtosis: {residuals.kurtosis():.3f}")
    
    # Tests de normalité (approximatif)
    from scipy.stats import jarque_bera
    try:
        jb_stat, jb_p = jarque_bera(residuals)
        print(f"  • Test de Jarque-Bera: p-value = {jb_p:.3f}")
        if jb_p > 0.05:
            print("    ✅ Résidus probablement normaux")
        else:
            print("    ⚠️ Résidus non-normaux détectés")
    except:
        print("  • Test de normalité non disponible")
    
    # Détection des outliers
    Q1 = residuals.quantile(0.25)
    Q3 = residuals.quantile(0.75)
    IQR = Q3 - Q1
    outlier_threshold = 1.5 * IQR
    outliers = residuals[(residuals < Q1 - outlier_threshold) | (residuals > Q3 + outlier_threshold)]
    
    print(f"\n🎯 DÉTECTION D'OUTLIERS:")
    print(f"  • Nombre d'outliers: {len(outliers)} ({len(outliers)/len(residuals)*100:.1f}%)")
    if len(outliers) > 0:
        print(f"  • Indices des outliers: {list(outliers.index)}")
    
    return residuals

def create_residual_visualizations(y_true, y_pred, residuals):
    """Crée des visualisations pour l'analyse des résidus"""
    
    print(f"\n📊 Création des visualisations...")
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Analyse des Résidus et Performances du Modèle', fontsize=16, fontweight='bold')
    
    # 1. Prédictions vs Réalité
    ax1 = axes[0, 0]
    ax1.scatter(y_true, y_pred, alpha=0.6, color='blue')
    min_val, max_val = min(y_true.min(), y_pred.min()), max(y_true.max(), y_pred.max())
    ax1.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Parfait')
    ax1.set_xlabel('Valeurs Réelles')
    ax1.set_ylabel('Prédictions')
    ax1.set_title('Prédictions vs Réalité')
    ax1.legend()
    ax1.grid(alpha=0.3)
    
    # 2. Distribution des résidus
    ax2 = axes[0, 1]
    ax2.hist(residuals, bins=15, color='lightblue', edgecolor='black', alpha=0.7)
    ax2.axvline(residuals.mean(), color='red', linestyle='--', label=f'Moyenne: {residuals.mean():.3f}')
    ax2.set_xlabel('Résidus')
    ax2.set_ylabel('Fréquence')
    ax2.set_title('Distribution des Résidus')
    ax2.legend()
    ax2.grid(axis='y', alpha=0.3)
    
    # 3. Résidus vs Prédictions
    ax3 = axes[0, 2]
    ax3.scatter(y_pred, residuals, alpha=0.6, color='green')
    ax3.axhline(0, color='red', linestyle='--')
    ax3.set_xlabel('Prédictions')
    ax3.set_ylabel('Résidus')
    ax3.set_title('Résidus vs Prédictions')
    ax3.grid(alpha=0.3)
    
    # 4. Q-Q Plot (approximatif)
    ax4 = axes[1, 0]
    from scipy.stats import probplot
    probplot(residuals, dist="norm", plot=ax4)
    ax4.set_title('Q-Q Plot (Normalité des Résidus)')
    ax4.grid(alpha=0.3)
    
    # 5. Erreurs absolues
    ax5 = axes[1, 1]
    abs_errors = np.abs(residuals)
    ax5.scatter(y_true, abs_errors, alpha=0.6, color='orange')
    ax5.set_xlabel('Valeurs Réelles')
    ax5.set_ylabel('Erreur Absolue')
    ax5.set_title('Erreurs Absolues vs Valeurs Réelles')
    ax5.grid(alpha=0.3)
    
    # 6. Résidus standardisés
    ax6 = axes[1, 2]
    standardized_residuals = residuals / residuals.std()
    ax6.scatter(range(len(standardized_residuals)), standardized_residuals, alpha=0.6, color='purple')
    ax6.axhline(0, color='red', linestyle='-', alpha=0.5)
    ax6.axhline(2, color='red', linestyle='--', alpha=0.5, label='±2σ')
    ax6.axhline(-2, color='red', linestyle='--', alpha=0.5)
    ax6.set_xlabel('Index')
    ax6.set_ylabel('Résidus Standardisés')
    ax6.set_title('Résidus Standardisés')
    ax6.legend()
    ax6.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"  ✅ Visualisations créées")

# Analyse des résidus si les prédictions sont disponibles
if 'predictions' in locals():
    residuals = residual_analysis(y_test, predictions)
    create_residual_visualizations(y_test, predictions, residuals)

# =====================================================================
# 4. VALIDATION CROISÉE ET ROBUSTESSE
# =====================================================================

print("\n" + "="*60)
print("🔄 VALIDATION CROISÉE ET TESTS DE ROBUSTESSE")
print("="*60)

def cross_validation_analysis(model, X, y, scaler, cv_folds=5):
    """Analyse par validation croisée"""
    
    print(f"\n🔄 Validation croisée avec {cv_folds} plis...")
    
    # Pour les modèles sklearn
    if hasattr(model, 'predict'):
        # Données normalisées pour la validation croisée
        X_scaled = scaler.transform(X)
        
        # Validation croisée pour différentes métriques
        cv_scores = {}
        
        # R²
        r2_scores = cross_val_score(model, X_scaled, y, cv=cv_folds, scoring='r2')
        cv_scores['R²'] = {'mean': r2_scores.mean(), 'std': r2_scores.std(), 'scores': r2_scores}
        
        # RMSE (négatif dans sklearn, donc on inverse)
        rmse_scores = cross_val_score(model, X_scaled, y, cv=cv_folds, scoring='neg_root_mean_squared_error')
        cv_scores['RMSE'] = {'mean': -rmse_scores.mean(), 'std': rmse_scores.std(), 'scores': -rmse_scores}
        
        # MAE (négatif dans sklearn, donc on inverse)
        mae_scores = cross_val_score(model, X_scaled, y, cv=cv_folds, scoring='neg_mean_absolute_error')
        cv_scores['MAE'] = {'mean': -mae_scores.mean(), 'std': mae_scores.std(), 'scores': -mae_scores}
        
        print(f"\n📊 RÉSULTATS DE LA VALIDATION CROISÉE:")
        for metric, results in cv_scores.items():
            print(f"  • {metric}:")
            print(f"    - Moyenne: {results['mean']:.3f}")
            print(f"    - Écart-type: {results['std']:.3f}")
            print(f"    - Scores individuels: {[f'{score:.3f}' for score in results['scores']]}")
        
        # Analyse de la stabilité
        r2_cv = cv_scores['R²']['std']
        if r2_cv < 0.05:
            print(f"\n✅ STABILITÉ EXCELLENTE: Écart-type R² = {r2_cv:.3f} < 0.05")
        elif r2_cv < 0.1:
            print(f"\n✅ STABILITÉ BONNE: Écart-type R² = {r2_cv:.3f} < 0.1")
        else:
            print(f"\n⚠️ STABILITÉ VARIABLE: Écart-type R² = {r2_cv:.3f} > 0.1")
        
        return cv_scores
    else:
        print("⚠️ Validation croisée non disponible pour ce type de modèle")
        return None

def learning_curve_analysis(model, X, y, scaler):
    """Analyse des courbes d'apprentissage"""
    
    print(f"\n📈 Analyse des courbes d'apprentissage...")
    
    if hasattr(model, 'predict'):
        X_scaled = scaler.transform(X)
        
        # Calcul des courbes d'apprentissage
        train_sizes = np.linspace(0.1, 1.0, 10)
        train_sizes_abs, train_scores, val_scores = learning_curve(
            model, X_scaled, y, train_sizes=train_sizes, cv=3, scoring='r2'
        )
        
        # Moyennes et écarts-types
        train_mean = np.mean(train_scores, axis=1)
        train_std = np.std(train_scores, axis=1)
        val_mean = np.mean(val_scores, axis=1)
        val_std = np.std(val_scores, axis=1)
        
        # Visualisation
        plt.figure(figsize=(10, 6))
        plt.plot(train_sizes_abs, train_mean, 'o-', color='blue', label='Score d\'entraînement')
        plt.fill_between(train_sizes_abs, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
        plt.plot(train_sizes_abs, val_mean, 'o-', color='red', label='Score de validation')
        plt.fill_between(train_sizes_abs, val_mean - val_std, val_mean + val_std, alpha=0.1, color='red')
        
        plt.xlabel('Taille de l\'échantillon d\'entraînement')
        plt.ylabel('Score R²')
        plt.title('Courbes d\'Apprentissage')
        plt.legend()
        plt.grid(alpha=0.3)
        plt.show()
        
        # Analyse de l'overfitting
        final_gap = train_mean[-1] - val_mean[-1]
        if final_gap < 0.05:
            print(f"✅ PAS D'OVERFITTING: Écart final = {final_gap:.3f}")
        elif final_gap < 0.1:
            print(f"⚠️ OVERFITTING LÉGER: Écart final = {final_gap:.3f}")
        else:
            print(f"❌ OVERFITTING IMPORTANT: Écart final = {final_gap:.3f}")
        
        return train_sizes_abs, train_mean, val_mean
    else:
        print("⚠️ Courbes d'apprentissage non disponibles pour ce type de modèle")
        return None, None, None

# Validation croisée et courbes d'apprentissage
if model is not None and X_test is not None:
    cv_results = cross_validation_analysis(model, X_test, y_test, scaler)
    learning_results = learning_curve_analysis(model, X_test, y_test, scaler)

# =====================================================================
# 5. ANALYSE D'IMPORTANCE ET INTERPRÉTABILITÉ
# =====================================================================

print("\n" + "="*60)
print("🔍 ANALYSE D'IMPORTANCE ET INTERPRÉTABILITÉ")
print("="*60)

def feature_importance_analysis(model, X, y, scaler, feature_names):
    """Analyse d'importance des features"""
    
    print(f"\n🔬 Analyse d'importance des features...")
    
    X_scaled = scaler.transform(X)
    
    # 1. Importance intégrée du modèle (si disponible)
    if hasattr(model, 'feature_importances_'):
        model_importance = model.feature_importances_
        importance_type = "Importance du modèle (basée sur l'arbre)"
    elif hasattr(model, 'coef_'):
        model_importance = np.abs(model.coef_)
        importance_type = "Coefficients absolus"
    else:
        model_importance = None
        importance_type = None
    
    # 2. Permutation importance (plus robuste)
    try:
        perm_importance = permutation_importance(model, X_scaled, y, n_repeats=5, random_state=42)
        perm_importance_mean = perm_importance.importances_mean
        perm_importance_std = perm_importance.importances_std
        
        print(f"✅ Permutation importance calculée")
        
        # DataFrame pour faciliter l'affichage
        importance_df = pd.DataFrame({
            'feature': feature_names,
            'perm_importance': perm_importance_mean,
            'perm_std': perm_importance_std
        })
        
        if model_importance is not None:
            importance_df['model_importance'] = model_importance
            
        importance_df = importance_df.sort_values('perm_importance', ascending=False)
        
        print(f"\n🏆 TOP 10 FEATURES PAR PERMUTATION IMPORTANCE:")
        for i, row in importance_df.head(10).iterrows():
            print(f"  {i+1:2d}. {row['feature']:25} | {row['perm_importance']:.4f} ± {row['perm_std']:.4f}")
        
        # Visualisation
        plt.figure(figsize=(12, 8))
        top_features = importance_df.head(15)
        
        if model_importance is not None:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
            
            # Permutation importance
            ax1.barh(range(len(top_features)), top_features['perm_importance'], 
                    xerr=top_features['perm_std'], color='skyblue', edgecolor='navy')
            ax1.set_yticks(range(len(top_features)))
            ax1.set_yticklabels(top_features['feature'])
            ax1.set_xlabel('Permutation Importance')
            ax1.set_title('Importance par Permutation')
            ax1.grid(axis='x', alpha=0.3)
            
            # Model importance
            ax2.barh(range(len(top_features)), top_features['model_importance'], 
                    color='lightgreen', edgecolor='darkgreen')
            ax2.set_yticks(range(len(top_features)))
            ax2.set_yticklabels(top_features['feature'])
            ax2.set_xlabel(importance_type)
            ax2.set_title(importance_type)
            ax2.grid(axis='x', alpha=0.3)
            
            plt.tight_layout()
        else:
            plt.barh(range(len(top_features)), top_features['perm_importance'], 
                    xerr=top_features['perm_std'], color='skyblue', edgecolor='navy')
            plt.yticks(range(len(top_features)), top_features['feature'])
            plt.xlabel('Permutation Importance')
            plt.title('Importance des Features par Permutation')
            plt.grid(axis='x', alpha=0.3)
        
        plt.gca().invert_yaxis()
        plt.show()
        
        return importance_df
        
    except Exception as e:
        print(f"⚠️ Erreur calcul permutation importance: {e}")
        return None

def shap_analysis(model, X_sample, scaler):
    """Analyse SHAP si disponible"""
    
    if not HAS_SHAP:
        print("⚠️ SHAP non disponible - analyse d'interprétabilité limitée")
        return None
    
    print(f"\n🔬 Analyse SHAP...")
    
    try:
        X_scaled = scaler.transform(X_sample)
        
        # Création de l'explainer SHAP
        if hasattr(model, 'predict_proba'):
            explainer = shap.Explainer(model, X_scaled)
        else:
            explainer = shap.Explainer(model.predict, X_scaled)
        
        # Calcul des valeurs SHAP pour un échantillon
        shap_values = explainer(X_scaled[:min(50, len(X_scaled))])
        
        # Visualisation summary plot
        shap.summary_plot(shap_values, X_sample.iloc[:min(50, len(X_sample))], show=False)
        plt.title('SHAP Summary Plot - Importance et Impact des Features')
        plt.show()
        
        # Feature importance globale
        feature_importance = np.abs(shap_values.values).mean(0)
        shap_importance_df = pd.DataFrame({
            'feature': X_sample.columns,
            'shap_importance': feature_importance
        }).sort_values('shap_importance', ascending=False)
        
        print(f"\n🏆 TOP 10 FEATURES PAR SHAP IMPORTANCE:")
        for i, row in shap_importance_df.head(10).iterrows():
            print(f"  {i+1:2d}. {row['feature']:25} | {row['shap_importance']:.4f}")
        
        return shap_values, shap_importance_df
        
    except Exception as e:
        print(f"⚠️ Erreur analyse SHAP: {e}")
        return None, None

# Analyses d'importance
if model is not None and X_test is not None:
    importance_df = feature_importance_analysis(model, X_test, y_test, scaler, X_test.columns.tolist())
    
    # Analyse SHAP sur un échantillon
    shap_values, shap_importance = shap_analysis(model, X_test.head(50), scaler)

# =====================================================================
# 6. ANALYSE PAR SEGMENTS
# =====================================================================

print("\n" + "="*60)
print("📊 ANALYSE PAR SEGMENTS")
print("="*60)

def segment_analysis(y_true, y_pred, X_test):
    """Analyse des performances par segments"""
    
    print(f"\n📊 Analyse par segments...")
    
    # Création de segments de risque
    risk_segments = pd.cut(y_true, bins=[0, 3, 6, 10], labels=['Faible', 'Moyen', 'Élevé'])
    
    # Performances par segment
    segment_results = {}
    
    for segment in risk_segments.cat.categories:
        mask = risk_segments == segment
        if mask.sum() > 0:
            y_true_seg = y_true[mask]
            y_pred_seg = y_pred[mask]
            
            segment_results[segment] = {
                'count': mask.sum(),
                'rmse': np.sqrt(mean_squared_error(y_true_seg, y_pred_seg)),
                'mae': mean_absolute_error(y_true_seg, y_pred_seg),
                'r2': r2_score(y_true_seg, y_pred_seg),
                'mape': np.mean(np.abs((y_true_seg - y_pred_seg) / y_true_seg)) * 100
            }
    
    print(f"\n📈 PERFORMANCES PAR SEGMENT DE RISQUE:")
    for segment, metrics in segment_results.items():
        print(f"\n  {segment} Risque ({metrics['count']} observations):")
        print(f"    - RMSE: {metrics['rmse']:.3f}")
        print(f"    - MAE: {metrics['mae']:.3f}")
        print(f"    - R²: {metrics['r2']:.3f}")
        print(f"    - MAPE: {metrics['mape']:.1f}%")
    
    # Visualisation par segments
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    segments = list(segment_results.keys())
    
    # RMSE par segment
    rmse_values = [segment_results[seg]['rmse'] for seg in segments]
    axes[0].bar(segments, rmse_values, color='lightblue', edgecolor='navy')
    axes[0].set_title('RMSE par Segment de Risque')
    axes[0].set_ylabel('RMSE')
    axes[0].grid(axis='y', alpha=0.3)
    
    # R² par segment
    r2_values = [segment_results[seg]['r2'] for seg in segments]
    axes[1].bar(segments, r2_values, color='lightgreen', edgecolor='darkgreen')
    axes[1].set_title('R² par Segment de Risque')
    axes[1].set_ylabel('R²')
    axes[1].grid(axis='y', alpha=0.3)
    
    # MAPE par segment
    mape_values = [segment_results[seg]['mape'] for seg in segments]
    axes[2].bar(segments, mape_values, color='lightcoral', edgecolor='darkred')
    axes[2].set_title('MAPE par Segment de Risque')
    axes[2].set_ylabel('MAPE (%)')
    axes[2].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return segment_results

# Analyse par segments
if 'predictions' in locals():
    segment_results = segment_analysis(y_test, predictions, X_test)

# =====================================================================
# 7. TESTS DE ROBUSTESSE
# =====================================================================

print("\n" + "="*60)
print("🛡️ TESTS DE ROBUSTESSE")
print("="*60)

def robustness_tests(model, X_test, y_test, scaler):
    """Tests de robustesse du modèle"""
    
    print(f"\n🛡️ Tests de robustesse...")
    
    X_scaled = scaler.transform(X_test)
    
    # 1. Test avec bruit ajouté
    noise_levels = [0.01, 0.05, 0.1, 0.2]
    noise_results = {}
    
    for noise_level in noise_levels:
        # Ajout de bruit gaussien
        X_noisy = X_scaled + np.random.normal(0, noise_level, X_scaled.shape)
        
        # Prédictions avec bruit
        if hasattr(model, 'predict'):
            y_pred_noisy = model.predict(X_noisy)
        else:
            y_pred_noisy = model.predict(X_noisy, verbose=0).flatten()
        
        # Métriques avec bruit
        noise_results[noise_level] = {
            'r2': r2_score(y_test, y_pred_noisy),
            'rmse': np.sqrt(mean_squared_error(y_test, y_pred_noisy))
        }
    
    print(f"\n🔊 ROBUSTESSE AU BRUIT:")
    for noise_level, metrics in noise_results.items():
        print(f"  Bruit {noise_level*100:3.0f}%: R² = {metrics['r2']:.3f}, RMSE = {metrics['rmse']:.3f}")
    
    # 2. Test avec valeurs manquantes simulées
    missing_percentages = [0.05, 0.1, 0.2]
    missing_results = {}
    
    for missing_pct in missing_percentages:
        X_missing = X_scaled.copy()
        
        # Simulation de valeurs manquantes (remplacées par la médiane)
        n_missing = int(missing_pct * X_missing.size)
        missing_indices = np.random.choice(X_missing.size, n_missing, replace=False)
        
        # Remplacement par la médiane de chaque feature
        for i in range(X_missing.shape[1]):
            col_missing = np.random.choice(X_missing.shape[0], int(missing_pct * X_missing.shape[0]), replace=False)
            X_missing[col_missing, i] = np.median(X_scaled[:, i])
        
        # Prédictions avec valeurs manquantes simulées
        if hasattr(model, 'predict'):
            y_pred_missing = model.predict(X_missing)
        else:
            y_pred_missing = model.predict(X_missing, verbose=0).flatten()
        
        missing_results[missing_pct] = {
            'r2': r2_score(y_test, y_pred_missing),
            'rmse': np.sqrt(mean_squared_error(y_test, y_pred_missing))
        }
    
    print(f"\n❓ ROBUSTESSE AUX VALEURS MANQUANTES:")
    for missing_pct, metrics in missing_results.items():
        print(f"  {missing_pct*100:3.0f}% manquant: R² = {metrics['r2']:.3f}, RMSE = {metrics['rmse']:.3f}")
    
    # Visualisation de la robustesse
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Robustesse au bruit
    noise_levels_pct = [n*100 for n in noise_levels]
    noise_r2 = [noise_results[n]['r2'] for n in noise_levels]
    ax1.plot(noise_levels_pct, noise_r2, 'o-', color='blue', linewidth=2, markersize=8)
    ax1.set_xlabel('Niveau de Bruit (%)')
    ax1.set_ylabel('R²')
    ax1.set_title('Robustesse au Bruit')
    ax1.grid(alpha=0.3)
    
    # Robustesse aux valeurs manquantes
    missing_pct_list = [m*100 for m in missing_percentages]
    missing_r2 = [missing_results[m]['r2'] for m in missing_percentages]
    ax2.plot(missing_pct_list, missing_r2, 'o-', color='red', linewidth=2, markersize=8)
    ax2.set_xlabel('Valeurs Manquantes (%)')
    ax2.set_ylabel('R²')
    ax2.set_title('Robustesse aux Valeurs Manquantes')
    ax2.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return noise_results, missing_results

# Tests de robustesse
if model is not None:
    noise_results, missing_results = robustness_tests(model, X_test, y_test, scaler)

# =====================================================================
# 8. GÉNÉRATION DU RAPPORT D'ÉVALUATION
# =====================================================================

print("\n" + "="*60)
print("📋 GÉNÉRATION DU RAPPORT D'ÉVALUATION")
print("="*60)

def generate_evaluation_report(model_metadata, performance_metrics, cv_results, importance_df, segment_results):
    """Génère un rapport complet d'évaluation"""
    
    print(f"\n📋 Génération du rapport...")
    
    report = {
        'evaluation_date': datetime.now().isoformat(),
        'model_info': {
            'name': model_metadata['model_name'],
            'type': model_metadata['model_type'],
            'training_date': model_metadata['model_training_date'],
            'features_count': model_metadata['num_features']
        },
        'performance_metrics': performance_metrics,
        'cross_validation': cv_results if cv_results else {},
        'feature_importance': importance_df.head(10).to_dict('records') if importance_df is not None else [],
        'segment_analysis': segment_results if 'segment_results' in locals() else {},
        'robustness': {
            'noise_test': noise_results if 'noise_results' in locals() else {},
            'missing_values_test': missing_results if 'missing_results' in locals() else {}
        }
    }
    
    # Interprétations et recommandations
    recommendations = []
    
    # Basé sur les performances générales
    if performance_metrics['R²'] > 0.8:
        recommendations.append("✅ Excellent modèle - Déploiement recommandé")
    elif performance_metrics['R²'] > 0.6:
        recommendations.append("✅ Bon modèle - Déploiement possible avec monitoring")
    else:
        recommendations.append("⚠️ Performances limitées - Amélioration nécessaire")
    
    # Basé sur la robustesse
    if 'noise_results' in locals():
        noise_degradation = (performance_metrics['R²'] - noise_results[0.1]['r2']) / performance_metrics['R²']
        if noise_degradation < 0.1:
            recommendations.append("✅ Modèle robuste au bruit")
        else:
            recommendations.append("⚠️ Sensibilité au bruit détectée")
    
    # Basé sur les segments
    if 'segment_results' in locals():
        segment_r2_values = [seg['r2'] for seg in segment_results.values()]
        if min(segment_r2_values) > 0.5:
            recommendations.append("✅ Performances cohérentes entre segments")
        else:
            recommendations.append("⚠️ Performances variables selon les segments de risque")
    
    report['recommendations'] = recommendations
    
    # Sauvegarde du rapport
    import json
    import os
    
    output_dir = '../models'
    os.makedirs(output_dir, exist_ok=True)
    
    report_filename = f"{output_dir}/evaluation_report.json"
    with open(report_filename, 'w') as f:
        json.dump(report, f, indent=2, default=str)
    
    print(f"✅ Rapport sauvegardé: {report_filename}")
    
    # Affichage du résumé
    print(f"\n📊 RÉSUMÉ DE L'ÉVALUATION:")
    print(f"  • Modèle évalué: {report['model_info']['name']}")
    print(f"  • Score R²: {performance_metrics['R²']:.3f}")
    print(f"  • RMSE: {performance_metrics['RMSE']:.3f}")
    print(f"  • MAPE: {performance_metrics['MAPE']:.1f}%")
    
    print(f"\n🎯 RECOMMANDATIONS:")
    for rec in recommendations:
        print(f"  {rec}")
    
    return report

# Génération du rapport final
if all(var in locals() for var in ['model_metadata', 'performance_metrics']):
    final_report = generate_evaluation_report(
        model_metadata, 
        performance_metrics,
        locals().get('cv_results'),
        locals().get('importance_df'),
        locals().get('segment_results')
    )

# =====================================================================
# 9. VISUALISATION GÉOSPATIALE (BONUS)
# =====================================================================

print("\n" + "="*60)
print("🗺️ VISUALISATION GÉOSPATIALE (OPTIONNEL)")
print("="*60)

def create_risk_map(X_test, y_test, predictions):
    """Crée une carte de risque géospatiale"""
    
    print(f"\n🗺️ Création de la carte de risque...")
    
    try:
        # Vérification des colonnes géographiques
        if 'latitude' in X_test.columns and 'longitude' in X_test.columns:
            
            # Centre de la carte (Boston)
            center_lat = X_test['latitude'].mean()
            center_lon = X_test['longitude'].mean()
            
            # Création de la carte
            m = folium.Map(location=[center_lat, center_lon], zoom_start=12)
            
            # Ajout des points avec couleurs selon le risque
            for idx, row in X_test.iterrows():
                lat, lon = row['latitude'], row['longitude']
                real_risk = y_test.iloc[idx] if idx < len(y_test) else 5
                pred_risk = predictions[idx] if idx < len(predictions) else 5
                
                # Couleur basée sur le risque prédit
                if pred_risk >= 7:
                    color = 'red'
                elif pred_risk >= 5:
                    color = 'orange'
                elif pred_risk >= 3:
                    color = 'yellow'
                else:
                    color = 'green'
                
                folium.CircleMarker(
                    location=[lat, lon],
                    radius=8,
                    popup=f"Risque réel: {real_risk:.2f}<br>Risque prédit: {pred_risk:.2f}",
                    fillColor=color,
                    color='black',
                    weight=1,
                    fillOpacity=0.7
                ).add_to(m)
            
            # Sauvegarde de la carte
            map_filename = '../models/risk_map.html'
            m.save(map_filename)
            print(f"✅ Carte sauvegardée: {map_filename}")
            
            return m
        else:
            print("⚠️ Colonnes géographiques non trouvées")
            return None
            
    except Exception as e:
        print(f"⚠️ Erreur création carte: {e}")
        return None

# Création de la carte si possible
if all(var in locals() for var in ['X_test', 'y_test', 'predictions']):
    risk_map = create_risk_map(X_test, y_test, predictions)

# =====================================================================
# 10. CONCLUSIONS ET PROCHAINES ÉTAPES
# =====================================================================

print("\n" + "="*60)
print("🎯 CONCLUSIONS ET PROCHAINES ÉTAPES")
print("="*60)

print(f"\n✅ ÉVALUATION COMPLÉTÉE:")

if 'performance_metrics' in locals():
    print(f"  • Performance globale: R² = {performance_metrics['R²']:.3f}")
    
    # Évaluation qualitative
    if performance_metrics['R²'] > 0.8:
        quality = "EXCELLENTE"
        emoji = "🏆"
    elif performance_metrics['R²'] > 0.6:
        quality = "BONNE"
        emoji = "✅"
    elif performance_metrics['R²'] > 0.4:
        quality = "MOYENNE"
        emoji = "⚠️"
    else:
        quality = "FAIBLE"
        emoji = "❌"
    
    print(f"  • Qualité du modèle: {emoji} {quality}")

print(f"\n📁 FICHIERS GÉNÉRÉS:")
generated_files = [
    '../models/evaluation_report.json',
    '../models/risk_map.html'
]

for filepath in generated_files:
    import os
    if os.path.exists(filepath):
        print(f"  ✅ {filepath}")
    else:
        print(f"  ⚠️ {filepath} (non généré)")

print(f"\n🚀 PROCHAINES ÉTAPES RECOMMANDÉES:")
if 'final_report' in locals() and 'recommendations' in final_report:
    for i, rec in enumerate(final_report['recommendations'], 1):
        print(f"  {i}. {rec}")

print(f"\n🔧 AMÉLIORATIONS POSSIBLES:")
print(f"  • Collecter plus de données d'entraînement")
print(f"  • Enrichir avec des features externes (météo, événements)")
print(f"  • Tester des architectures de modèles plus complexes")
print(f"  • Implémenter un système de re-entraînement automatique")
print(f"  • Développer des alertes en temps réel")

print("\n" + "="*60)
print("✨ ÉVALUATION DU MODÈLE TERMINÉE AVEC SUCCÈS")
print("="*60)

⚠️ SHAP non installé - analyse d'interprétabilité limitée
📊 Début de l'évaluation du modèle

📁 Chargement du modèle et des données...
❌ Fichier non trouvé: [Errno 2] No such file or directory: '../models/model_metadata.json'
   Vérifiez que le model development a été exécuté

📈 ÉVALUATION DES PERFORMANCES GÉNÉRALES

🔍 ANALYSE DES RÉSIDUS ET ERREURS

🔄 VALIDATION CROISÉE ET TESTS DE ROBUSTESSE

🔍 ANALYSE D'IMPORTANCE ET INTERPRÉTABILITÉ

📊 ANALYSE PAR SEGMENTS

🛡️ TESTS DE ROBUSTESSE

📋 GÉNÉRATION DU RAPPORT D'ÉVALUATION

🗺️ VISUALISATION GÉOSPATIALE (OPTIONNEL)

🎯 CONCLUSIONS ET PROCHAINES ÉTAPES

✅ ÉVALUATION COMPLÉTÉE:

📁 FICHIERS GÉNÉRÉS:
  ⚠️ ../models/evaluation_report.json (non généré)
  ⚠️ ../models/risk_map.html (non généré)

🚀 PROCHAINES ÉTAPES RECOMMANDÉES:

🔧 AMÉLIORATIONS POSSIBLES:
  • Collecter plus de données d'entraînement
  • Enrichir avec des features externes (météo, événements)
  • Tester des architectures de modèles plus complexes
  • Implémenter un système de re-e